## Import Libraries

In [1]:
import dill
import pandas as pd
from catboost import CatBoostClassifier
from datetime import datetime
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBClassifier

## Create Dataframe

In [2]:
#начал работать с датасетами по отдельности, потому что для слияния оригиналов не хватает оперативной памяти

In [3]:
#загрузка датасета
df_hits_original = pd.read_csv("ga_hits.csv")

In [4]:
#рабочая копия датафрейма
df_hits = df_hits_original.copy()
df_hits.head()

,session_id,hit_date,hit_time,hit_number,hit_type,hit_referer,hit_page_path,event_category,event_action,event_label,event_value
0,5639623078712724064.1640254056.1640254056,2021-12-23,597864.0,30,event,NaN,sberauto.com/cars?utm_source_initial=google&ut...,quiz,quiz_show,NaN,NaN
1,7750352294969115059.1640271109.1640271109,2021-12-23,597331.0,41,event,NaN,sberauto.com/cars/fiat?city=1&city=18&rental_c...,quiz,quiz_show,NaN,NaN
2,885342191847998240.1640235807.1640235807,2021-12-23,796252.0,49,event,NaN,sberauto.com/cars/all/volkswagen/polo/e994838f...,quiz,quiz_show,NaN,NaN
3,142526202120934167.1640211014.1640211014,2021-12-23,934292.0,46,event,NaN,sberauto.com/cars?utm_source_initial=yandex&ut...,quiz,quiz_show,NaN,NaN
4,3450086108837475701.1640265078.1640265078,2021-12-23,768741.0,79,event,NaN,sberauto.com/cars/all/mercedes-benz/cla-klasse...,quiz,quiz_show,NaN,NaN


In [5]:
#размерность датафрейма
df_hits.shape

(15726470, 11)

In [6]:
#проверка на наличие дублей в данных
df_hits.duplicated().unique()

array([False])

In [7]:
#ознакомление с количеством уникальных значений в колонках
df_hits.nunique()

session_id        1734610
hit_date              226
hit_time           925887
hit_number            500
hit_type                1
hit_referer         37873
hit_page_path      342715
event_category         52
event_action          230
event_label         39825
event_value             0
dtype: int64

In [8]:
#удаление явно неинформативных колонок
df_hits = df_hits.drop(["hit_number", "hit_type", "hit_page_path", "event_category", "event_value"], axis=1)
df_hits.head()

,session_id,hit_date,hit_time,hit_referer,event_action,event_label
0,5639623078712724064.1640254056.1640254056,2021-12-23,597864.0,NaN,quiz_show,NaN
1,7750352294969115059.1640271109.1640271109,2021-12-23,597331.0,NaN,quiz_show,NaN
2,885342191847998240.1640235807.1640235807,2021-12-23,796252.0,NaN,quiz_show,NaN
3,142526202120934167.1640211014.1640211014,2021-12-23,934292.0,NaN,quiz_show,NaN
4,3450086108837475701.1640265078.1640265078,2021-12-23,768741.0,NaN,quiz_show,NaN


In [9]:
#загрузка датасета
df_sessions_original = pd.read_csv("ga_sessions.csv", dtype={"client_id":"str"})

In [10]:
#рабочая копия датафрейма
df_sessions = df_sessions_original.copy()
df_sessions.head()

,session_id,client_id,visit_date,visit_time,visit_number,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,device_brand,device_model,device_screen_resolution,device_browser,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2108382700.1637753791,2021-11-24,14:36:32,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,210838531.1636867288,2021-11-14,08:21:30,1,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Samsung,NaN,385x854,Samsung Internet,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2108385331.1640648523,2021-12-28,02:42:06,1,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Huawei,NaN,360x720,Chrome,Russia,Krasnoyarsk
3,9055447046360770272.1622255328.1622255328,2108385564.1622255328,2021-05-29,05:00:00,1,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2108385564.1622255328,2021-05-29,05:00:00,2,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,NaN,Xiaomi,NaN,393x786,Chrome,Russia,Moscow


In [11]:
#размерность датафрейма
df_sessions.shape

(1860042, 18)

In [12]:
#проверка на наличие дублей в данных
df_sessions.duplicated().unique()

array([False])

In [13]:
#удаление явно неинформативных колонок
df_sessions = df_sessions.drop(["client_id", "visit_number", "device_brand", "device_model", "device_screen_resolution", "device_browser"], axis=1)
df_sessions.head()

,session_id,visit_date,visit_time,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,geo_country,geo_city
0,9055434745589932991.1637753792.1637753792,2021-11-24,14:36:32,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Russia,Zlatoust
1,905544597018549464.1636867290.1636867290,2021-11-14,08:21:30,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Russia,Moscow
2,9055446045651783499.1640648526.1640648526,2021-12-28,02:42:06,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Russia,Krasnoyarsk
3,9055447046360770272.1622255328.1622255328,2021-05-29,05:00:00,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,NaN,Russia,Moscow
4,9055447046360770272.1622255345.1622255345,2021-05-29,05:00:00,kjsLglQLzykiRbcDiGcD,cpc,NaN,NaN,NaN,mobile,NaN,Russia,Moscow


In [14]:
#слияние датафреймов
df_full = df_sessions.merge(df_hits, how="left", on="session_id")
df_full.head()

,session_id,visit_date,visit_time,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,geo_country,geo_city,hit_date,hit_time,hit_referer,event_action,event_label
0,9055434745589932991.1637753792.1637753792,2021-11-24,14:36:32,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Russia,Zlatoust,2021-11-24,3665.0,NaN,sub_landing,NaN
1,9055434745589932991.1637753792.1637753792,2021-11-24,14:36:32,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Russia,Zlatoust,2021-11-24,46592.0,NaN,sub_view_cars_click,vodKSlUobUWTVlgsJqdI
2,905544597018549464.1636867290.1636867290,2021-11-14,08:21:30,MvfHsxITijuriZxsqZqt,cpm,FTjNLDyTrXaWYgZymFkV,xhoenQgDQsgfEPYNPwKO,IGUCNvHlhfHpROGclCit,mobile,Android,Russia,Moscow,2021-11-14,921.0,NaN,sub_landing,NaN
3,9055446045651783499.1640648526.1640648526,2021-12-28,02:42:06,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Russia,Krasnoyarsk,2021-12-28,84155.0,NaN,search_form_region,KWTCzSIXzoqUWjfUQMgP
4,9055446045651783499.1640648526.1640648526,2021-12-28,02:42:06,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Russia,Krasnoyarsk,2021-12-28,102979.0,NaN,search_form_region,CBqnBQaKoQUyWJhLcxxN


In [15]:
print (f"Размерность датафреймов:\ndf_hits: {df_hits.shape}\ndf_sessions: {df_sessions.shape}\ndf_full: {df_full.shape}")

Размерность датафреймов:
df_hits: (15726470, 6)
df_sessions: (1860042, 12)
df_full: (15812995, 17)


## Data Preparation

### Data Cleaning

In [16]:
#проверка на наличие дублей в данных
df_full.duplicated().unique()

array([False,  True])

In [17]:
df_full[df_full.duplicated()]

,session_id,visit_date,visit_time,utm_source,utm_medium,utm_campaign,utm_adcontent,utm_keyword,device_category,device_os,geo_country,geo_city,hit_date,hit_time,hit_referer,event_action,event_label
20,9055447046360770272.1622255328.1622255328,2021-05-29,05:00:00,kjsLglQLzykiRbcDiGcD,cpc,NaN,NOBKLgtuvqYWkXQHeYWM,NaN,mobile,NaN,Russia,Moscow,2021-05-29,NaN,HbolMJUevblAbkHClEQa,sub_landing,KclpemfoHstknWHFiLit
27,9055447192389856083.1622453074.1622453074,2021-05-31,12:00:00,kjsLglQLzykiRbcDiGcD,organic,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Russia,Saint Petersburg,2021-05-31,NaN,HbolMJUevblAbkHClEQa,view_card,KclpemfoHstknWHFiLit
28,9055447192389856083.1622453074.1622453074,2021-05-31,12:00:00,kjsLglQLzykiRbcDiGcD,organic,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Russia,Saint Petersburg,2021-05-31,NaN,HbolMJUevblAbkHClEQa,view_card,KclpemfoHstknWHFiLit
29,9055447192389856083.1622453074.1622453074,2021-05-31,12:00:00,kjsLglQLzykiRbcDiGcD,organic,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Russia,Saint Petersburg,2021-05-31,NaN,HbolMJUevblAbkHClEQa,view_card,KclpemfoHstknWHFiLit
30,9055447192389856083.1622453074.1622453074,2021-05-31,12:00:00,kjsLglQLzykiRbcDiGcD,organic,LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Russia,Saint Petersburg,2021-05-31,NaN,HbolMJUevblAbkHClEQa,view_card,KclpemfoHstknWHFiLit
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15812908,9055394342833425189.1638599463.1638599463,2021-12-04,09:31:03,ZpYIoDJMcFzVoPFsHGJL,banner,LEoPHuyFvzoNfnzGgfcd,vCIpmpaGBnIQhyYNkXqp,puhZPIYqKXeFPaUviSjo,mobile,Android,Russia,Birsk,2021-12-04,74041.0,NaN,view_card,NaN
15812932,9055421130527858185.1622007305.1622007305,2021-05-26,08:00:00,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Russia,Stavropol,2021-05-26,NaN,HbolMJUevblAbkHClEQa,view_card,KclpemfoHstknWHFiLit
15812933,9055421130527858185.1622007305.1622007305,2021-05-26,08:00:00,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Russia,Stavropol,2021-05-26,NaN,HbolMJUevblAbkHClEQa,view_card,KclpemfoHstknWHFiLit
15812937,9055421130527858185.1622007305.1622007305,2021-05-26,08:00:00,fDLlAcSmythWSCVMvqvL,(none),LTuZkdKfxRGVceoWkVyg,JNHcPlZPxEMWDnRiyoBf,NaN,mobile,NaN,Russia,Stavropol,2021-05-26,NaN,HbolMJUevblAbkHClEQa,view_new_card,KclpemfoHstknWHFiLit


In [18]:
df_full.drop_duplicates(inplace=True)
df_full.shape

(10905054, 17)

In [19]:
#рабочая копия датафрейма
df = df_full.copy()

In [20]:
#удаление колонки с идентификаторами
df = df.drop(["session_id"], axis=1)

In [21]:
print(f"Количество полностью заполненных объектов из всей выборки: {len(df.dropna())}\
\nПроцент полностью заполненных объектов из всей выборки: {round(len(df.dropna()) / len(df) * 100, 2)}")

Количество полностью заполненных объектов из всей выборки: 100958
Процент полностью заполненных объектов из всей выборки: 0.93


In [22]:
missing_values = ((df.isna().sum() / len(df)) * 100).sort_values(ascending = False)
print(f"Процент пропущенных значений: \n{missing_values}")

Процент пропущенных значений: 
hit_referer        57.942308
utm_keyword        40.979293
hit_time           40.592454
device_os          40.512317
event_label        34.935618
utm_adcontent      16.740101
utm_campaign       13.239164
hit_date            1.171714
event_action        1.171714
utm_source          0.006502
visit_date          0.000000
visit_time          0.000000
utm_medium          0.000000
device_category     0.000000
geo_country         0.000000
geo_city            0.000000
dtype: float64


In [26]:
#список колонок с большим количеством пропусков в данных
columns_for_drop = []

for col_name, data in df.items():
    if data.isna().sum() / len(df) * 100 >= 1.5:
        columns_for_drop.append(col_name)
        
columns_for_drop

['utm_campaign',
 'utm_adcontent',
 'utm_keyword',
 'device_os',
 'hit_time',
 'hit_referer',
 'event_label']

In [27]:
#удаление колонок с большим количеством пропусков в данных
df = df.drop(columns_for_drop, axis=1)
df.head()

,visit_date,visit_time,utm_source,utm_medium,device_category,geo_country,geo_city,hit_date,event_action
0,2021-11-24,14:36:32,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Russia,Zlatoust,2021-11-24,sub_landing
1,2021-11-24,14:36:32,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Russia,Zlatoust,2021-11-24,sub_view_cars_click
2,2021-11-14,08:21:30,MvfHsxITijuriZxsqZqt,cpm,mobile,Russia,Moscow,2021-11-14,sub_landing
3,2021-12-28,02:42:06,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Russia,Krasnoyarsk,2021-12-28,search_form_region
4,2021-12-28,02:42:06,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Russia,Krasnoyarsk,2021-12-28,search_form_region


In [28]:
missing_values = ((df.isna().sum() / len(df)) * 100).sort_values(ascending = False)
print(f"Процент пропущенных значений: \n{missing_values}")

Процент пропущенных значений: 
hit_date           1.171714
event_action       1.171714
utm_source         0.006502
visit_date         0.000000
visit_time         0.000000
utm_medium         0.000000
device_category    0.000000
geo_country        0.000000
geo_city           0.000000
dtype: float64


In [29]:
#удаление строк с пропусками данных в колонке "event_action" (целевыми действиями)
df = df[~(df["event_action"].isna())]

In [30]:
#ознакомление со строками, в которых в "utm_source" пропущены данные
df.loc[df["utm_source"].isna()]

,visit_date,visit_time,utm_source,utm_medium,device_category,geo_country,geo_city,hit_date,event_action
660768,2021-12-29,00:50:07,NaN,(not set),mobile,Russia,Saint Petersburg,2021-12-29,sub_landing
660769,2021-12-29,00:50:07,NaN,(not set),mobile,Russia,Saint Petersburg,2021-12-29,sub_view_cars_click
1041133,2021-11-18,18:42:25,NaN,Sbol_catalog,mobile,Russia,Balashikha,2021-11-18,quiz_show
1152796,2021-11-22,18:30:59,NaN,(not set),mobile,Russia,Moscow,2021-11-22,view_new_card
1152797,2021-11-22,18:30:59,NaN,(not set),mobile,Russia,Moscow,2021-11-22,view_card
...,...,...,...,...,...,...,...,...,...
14734618,2021-11-21,04:53:43,NaN,(not set),mobile,Russia,Novosibirsk,2021-11-21,sub_landing
14734619,2021-11-21,04:53:43,NaN,(not set),mobile,Russia,Novosibirsk,2021-11-21,sub_landing
15071323,2021-11-13,15:34:23,NaN,(not set),mobile,Russia,Yuzhno-Sakhalinsk,2021-11-13,go_to_offer
15071324,2021-11-13,15:34:23,NaN,(not set),mobile,Russia,Yuzhno-Sakhalinsk,2021-11-13,sub_landing


In [32]:
#заполнение пропусков в колонке "utm_source" наиболее часто встречающимися значениями
df["utm_source"] = df["utm_source"].fillna(df["utm_source"].mode()[0])

In [34]:
print(f"Количество полностью заполненных объектов из всей выборки: {len(df.dropna())}\
\nПроцент полностью заполненных объектов из всей выборки: {round(len(df.dropna()) / len(df) * 100, 2)}")

Количество полностью заполненных объектов из всей выборки: 10777278
Процент полностью заполненных объектов из всей выборки: 100.0


In [35]:
#в датафрейме есть замаскированные пропуски - значения в скобках - поиск
set_not = set()

for i in range (len(list(df.columns))):
    for col_name, data in df[list(df.columns)[i]].items():
        if data[0] == "(":
            set_not.add(list[list(df.columns)[i], data])
set_not

{list['geo_city', '(not set)'],
 list['geo_country', '(not set)'],
 list['utm_medium', '(none)'],
 list['utm_medium', '(not set)']}

In [36]:
print(f'доля значений "(none)" и "(not set)" в колонке "utm_medium" - {round(df["utm_medium"].loc[(df["utm_medium"] == "(none)") | (df["utm_medium"] == "(not set)")].count() / df["utm_medium"].count() * 100, 2)} %')

доля значений "(none)" и "(not set)" в колонке "utm_medium" - 17.15 %


In [37]:
#удаление колонки "utm_medium" с большим количеством замаскированных пропусков в данных
df = df.drop("utm_medium", axis=1)
df.head()

,visit_date,visit_time,utm_source,device_category,geo_country,geo_city,hit_date,event_action
0,2021-11-24,14:36:32,ZpYIoDJMcFzVoPFsHGJL,mobile,Russia,Zlatoust,2021-11-24,sub_landing
1,2021-11-24,14:36:32,ZpYIoDJMcFzVoPFsHGJL,mobile,Russia,Zlatoust,2021-11-24,sub_view_cars_click
2,2021-11-14,08:21:30,MvfHsxITijuriZxsqZqt,mobile,Russia,Moscow,2021-11-14,sub_landing
3,2021-12-28,02:42:06,ZpYIoDJMcFzVoPFsHGJL,mobile,Russia,Krasnoyarsk,2021-12-28,search_form_region
4,2021-12-28,02:42:06,ZpYIoDJMcFzVoPFsHGJL,mobile,Russia,Krasnoyarsk,2021-12-28,search_form_region


In [38]:
print(f'доля значений "(not set)" в колонке "geo_country" - {round(df["geo_country"].loc[df["geo_country"] == "(not set)"].count() / df["geo_country"].count() * 100, 2)}%')

доля значений "(not set)" в колонке "geo_country" - 0.06%


In [39]:
#ознакомление с уникальными значениями в колонке "geo_country"
df["geo_country"].value_counts()

Russia                 10500823
Ukraine                   53506
United States             23956
Belarus                   18741
Germany                   14952
                         ...   
Congo - Brazzaville           2
Barbados                      1
Paraguay                      1
Cameroon                      1
Puerto Rico                   1
Name: geo_country, Length: 159, dtype: int64

In [40]:
print(f'доля значений, отличных от "Russia", в колонке "geo_country" - {round(df["geo_country"].loc[df["geo_country"] != "Russia"].count() / df["geo_country"].count() * 100, 2)}%')

доля значений, отличных от "Russia", в колонке "geo_country" - 2.57%


In [41]:
#замена большого количества редко встречающихся значений в колонке "geo_country" на "other"
df["geo_country"].loc[df["geo_country"] != "Russia"] = "other"
df["geo_country"].value_counts()

Russia    10500823
other       276455
Name: geo_country, dtype: int64

In [42]:
print(f'доля значений "(not set)" в колонке "geo_city" - {round(df["geo_city"].loc[df["geo_city"] == "(not set)"].count() / df["geo_country"].count() * 100, 2)}%')

доля значений "(not set)" в колонке "geo_city" - 3.71%


In [43]:
#ознакомление с уникальными значениями в колонке "geo_city"
df["geo_city"].value_counts()

Moscow              4420572
Saint Petersburg    1596446
(not set)            399943
Yekaterinburg        250340
Krasnodar            227215
                     ...   
Constantine               1
Sandy Springs             1
Catanzaro                 1
Southampton               1
Yavoriv                   1
Name: geo_city, Length: 2389, dtype: int64

In [44]:
#список значений в колонке "geo_city", составляющих не менее 1% всех значений, исключая замаскированные пропуски
cities_list = list(zip(list(df["geo_city"].unique()), list(df["geo_city"].value_counts(sort=False))))
significant_cities = []
for elem in cities_list:
    if elem[1] / len (df["geo_city"]) >= 0.01 and elem[0] != "(not set)":
        significant_cities.append(elem[0])
significant_cities

['Moscow',
 'Krasnoyarsk',
 'Saint Petersburg',
 'Kazan',
 'Yekaterinburg',
 'Samara',
 'Chelyabinsk',
 'Krasnodar',
 'Nizhny Novgorod',
 'Ufa',
 'Novosibirsk']

In [45]:
#замена редких значений в колонке "geo_city" (каждое составляет менее 1% всех значений) и замаскированных пропусков на "other"
df["geo_city"].loc[
    (df["geo_city"] != "Moscow") &
    (df["geo_city"] != "Krasnoyarsk") &
    (df["geo_city"] != "Saint Petersburg") & 
    (df["geo_city"] != "Kazan") & 
    (df["geo_city"] != "Yekaterinburg") & 
    (df["geo_city"] != "Samara") & 
    (df["geo_city"] != "Chelyabinsk") & 
    (df["geo_city"] != "Krasnodar") & 
    (df["geo_city"] != "Nizhny Novgorod") & 
    (df["geo_city"] != "Ufa") & 
    (df["geo_city"] != "Novosibirsk")
    ] = "other"

In [46]:
#ознакомление с уникальными значениями в колонке "geo_country" после редактирования
df["geo_city"].value_counts()

Moscow              4420572
other               3244640
Saint Petersburg    1596446
Yekaterinburg        250340
Krasnodar            227215
Kazan                202077
Samara               172956
Ufa                  150973
Nizhny Novgorod      146346
Novosibirsk          146272
Chelyabinsk          110634
Krasnoyarsk          108807
Name: geo_city, dtype: int64

In [47]:
#ознакомление с уникальными значениями в колонке "utm_source"
df["utm_source"].value_counts()

ZpYIoDJMcFzVoPFsHGJL    4222259
fDLlAcSmythWSCVMvqvL    1845461
kjsLglQLzykiRbcDiGcD    1158773
BHcvLfOaCWvWTykYqHVe     789792
bByPQxmDaMXgpHeypKSM     703675
                         ...   
jqlUOdZBNZYfInQVcZlS          1
VdeFdoGCqZBxFfKSHNLl          1
xlrxvewIthVdRefMQCED          1
XGOsamgBesNClOIokPJY          1
cdPAGAvKeBGoqAkQBuGR          1
Name: utm_source, Length: 280, dtype: int64

In [48]:
#список значений в колонке "utm_source", составляющих не менее 1% всех значений
utm_source = list(zip(list(df["utm_source"].unique()), list(df["utm_source"].value_counts(sort=False))))
significant_utm_source = []
for elem in utm_source:
    if elem[1] / len (df["utm_source"]) >= 0.01:
        significant_utm_source.append(elem[0])
significant_utm_source

['ZpYIoDJMcFzVoPFsHGJL',
 'MvfHsxITijuriZxsqZqt',
 'kjsLglQLzykiRbcDiGcD',
 'fDLlAcSmythWSCVMvqvL',
 'BHcvLfOaCWvWTykYqHVe',
 'vFcAhRxLfOWKhvxjELkx',
 'QxAxdyPLuQMEcrdZWdWb',
 'RmEBuqrriAfAVsLQQmhk',
 'bByPQxmDaMXgpHeypKSM',
 'jaSOmLICuBzCFqHfBdRg',
 'aXQzDWsJuGXeBXexNHjc']

In [49]:
#замена редких значений в колонке "utm_source" (каждое составляет менее 1% всех значений) на "other"
df["utm_source"].loc[
    (df["utm_source"] != "ZpYIoDJMcFzVoPFsHGJL") & 
    (df["utm_source"] != "MvfHsxITijuriZxsqZqt") & 
    (df["utm_source"] != "kjsLglQLzykiRbcDiGcD") & 
    (df["utm_source"] != "fDLlAcSmythWSCVMvqvL") & 
    (df["utm_source"] != "BHcvLfOaCWvWTykYqHVe") & 
    (df["utm_source"] != "vFcAhRxLfOWKhvxjELkx") & 
    (df["utm_source"] != "QxAxdyPLuQMEcrdZWdWb") & 
    (df["utm_source"] != "RmEBuqrriAfAVsLQQmhk") & 
    (df["utm_source"] != "bByPQxmDaMXgpHeypKSM") & 
    (df["utm_source"] != "jaSOmLICuBzCFqHfBdRg") & 
    (df["utm_source"] != "aXQzDWsJuGXeBXexNHjc")
    ] = "other"

In [50]:
#ознакомление с уникальными значениями в колонке "utm_source" после редактирования
df["utm_source"].value_counts()

ZpYIoDJMcFzVoPFsHGJL    4222259
fDLlAcSmythWSCVMvqvL    1845461
kjsLglQLzykiRbcDiGcD    1158773
BHcvLfOaCWvWTykYqHVe     789792
bByPQxmDaMXgpHeypKSM     703675
other                    605554
MvfHsxITijuriZxsqZqt     449150
QxAxdyPLuQMEcrdZWdWb     284689
aXQzDWsJuGXeBXexNHjc     202632
jaSOmLICuBzCFqHfBdRg     178555
vFcAhRxLfOWKhvxjELkx     177640
RmEBuqrriAfAVsLQQmhk     159098
Name: utm_source, dtype: int64

### Feature Engineering

In [51]:
#ознакомление с типами данных в датафрейме
df.dtypes

visit_date         object
visit_time         object
utm_source         object
device_category    object
geo_country        object
geo_city           object
hit_date           object
event_action       object
dtype: object

In [52]:
#приведение значений - дат к соответствующему типу данных
df["visit_date"] = pd.to_datetime(df["visit_date"])
df["hit_date"] = pd.to_datetime(df["hit_date"])

In [53]:
#контроль изменения типов данных
df.dtypes

visit_date         datetime64[ns]
visit_time                 object
utm_source                 object
device_category            object
geo_country                object
geo_city                   object
hit_date           datetime64[ns]
event_action               object
dtype: object

In [54]:
#создание новых признаков на основе дат
df["visit_month"] = df["visit_date"].apply(lambda x: x.month)
df["visit_day_of_week"] = df["visit_date"].dt.dayofweek
df["hit_month"] = df["hit_date"].apply(lambda x: x.month)
df["hit_day_of_week"] = df["hit_date"].dt.dayofweek

In [55]:
#изменения типа данных в колонках с целью оптимизации использования оперативной памяти
df["visit_month"] = df["visit_month"].astype("int8")
df["visit_day_of_week"] = df["visit_day_of_week"].astype("int8")
df["hit_month"] = df["hit_month"].astype("int8")
df["hit_day_of_week"] = df["hit_day_of_week"].astype("int8")

In [56]:
#удаление неактуальных колонок
#удаляю сразу с целью оптимизации использования оперативной памяти
df = df.drop(["visit_date", "visit_time", "hit_date"], axis=1)
df.head()

,utm_source,device_category,geo_country,geo_city,event_action,visit_month,visit_day_of_week,hit_month,hit_day_of_week
0,ZpYIoDJMcFzVoPFsHGJL,mobile,Russia,other,sub_landing,11,2,11,2
1,ZpYIoDJMcFzVoPFsHGJL,mobile,Russia,other,sub_view_cars_click,11,2,11,2
2,MvfHsxITijuriZxsqZqt,mobile,Russia,Moscow,sub_landing,11,6,11,6
3,ZpYIoDJMcFzVoPFsHGJL,mobile,Russia,Krasnoyarsk,search_form_region,12,1,12,1
4,ZpYIoDJMcFzVoPFsHGJL,mobile,Russia,Krasnoyarsk,search_form_region,12,1,12,1


In [57]:
#список колонок - категориальных признаков с отображением количества категорий
col_names = ["utm_source", "device_category", "geo_country", "geo_city"]
for col_name, data in df.items():
    for i in range (len(col_names)):
        if col_name == col_names[i]:
            print (f"{col_name} - {data.nunique()}")

utm_source - 12
device_category - 3
geo_country - 2
geo_city - 12


In [58]:
#выделение колонок - категориальных признаков в отдельный датафрейм с целью преобразования их значений для обучения моделей
df_cat = df[col_names]
df_cat.head()

,utm_source,device_category,geo_country,geo_city
0,ZpYIoDJMcFzVoPFsHGJL,mobile,Russia,other
1,ZpYIoDJMcFzVoPFsHGJL,mobile,Russia,other
2,MvfHsxITijuriZxsqZqt,mobile,Russia,Moscow
3,ZpYIoDJMcFzVoPFsHGJL,mobile,Russia,Krasnoyarsk
4,ZpYIoDJMcFzVoPFsHGJL,mobile,Russia,Krasnoyarsk


In [59]:
#преобразование значений
ohe = OneHotEncoder(sparse=False, dtype="int8")
ohe.fit(df_cat[col_names])
ohe_data = ohe.transform(df_cat[col_names])
print (f"{ohe_data} \n\nразмерность: {ohe_data.shape}")

[[0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]] 

размерность: (10777278, 29)


In [60]:
#создание новых признаков
df[ohe.get_feature_names_out()] = ohe_data
df.head()

,utm_source,device_category,geo_country,geo_city,event_action,visit_month,visit_day_of_week,hit_month,hit_day_of_week,utm_source_BHcvLfOaCWvWTykYqHVe,...,geo_city_Krasnodar,geo_city_Krasnoyarsk,geo_city_Moscow,geo_city_Nizhny Novgorod,geo_city_Novosibirsk,geo_city_Saint Petersburg,geo_city_Samara,geo_city_Ufa,geo_city_Yekaterinburg,geo_city_other
0,ZpYIoDJMcFzVoPFsHGJL,mobile,Russia,other,sub_landing,11,2,11,2,0,...,0,0,0,0,0,0,0,0,0,1
1,ZpYIoDJMcFzVoPFsHGJL,mobile,Russia,other,sub_view_cars_click,11,2,11,2,0,...,0,0,0,0,0,0,0,0,0,1
2,MvfHsxITijuriZxsqZqt,mobile,Russia,Moscow,sub_landing,11,6,11,6,0,...,0,0,1,0,0,0,0,0,0,0
3,ZpYIoDJMcFzVoPFsHGJL,mobile,Russia,Krasnoyarsk,search_form_region,12,1,12,1,0,...,0,1,0,0,0,0,0,0,0,0
4,ZpYIoDJMcFzVoPFsHGJL,mobile,Russia,Krasnoyarsk,search_form_region,12,1,12,1,0,...,0,1,0,0,0,0,0,0,0,0


In [61]:
#создание целевого признака для обучения моделей на основе колонки "event_action"
df["target"] = df["event_action"]

In [62]:
#преобразование значений: "1" - совершение любого целевого действия
df["target"].loc[
    (df["target"] == "sub_car_claim_click") | 
    (df["target"] == "sub_car_claim_submit_click") | 
    (df["target"] == "sub_open_dialog_click") | 
    (df["target"] == "sub_custom_question_submit_click") | 
    (df["target"] == "sub_call_number_click") | 
    (df["target"] == "sub_callback_submit_click") | 
    (df["target"] == "sub_submit_success") | 
    (df["target"] == "sub_car_request_submit_click")
    ] = 1

C:\Users\Дмитрий\AppData\Local\Temp\ipykernel_11044\4270629084.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["target"].loc[


In [63]:
#преобразование значений: "0" - несовершение целевого действия
df["target"].loc[df["target"] != 1] = 0

C:\Users\Дмитрий\AppData\Local\Temp\ipykernel_11044\806002489.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["target"].loc[df["target"] != 1] = 0


In [64]:
#контроль изменений в колонке "target"
df["target"].value_counts()

0    10682195
1       95083
Name: target, dtype: int64

In [65]:
#изменения типа данных в колонке "target" с целью оптимизации использования оперативной памяти
df["target"] = df["target"].astype("int8")

In [66]:
#удаление неактуальных колонок
df = df.drop(["utm_source", "device_category", "geo_country", "geo_city", "event_action"], axis=1)
df.head()

,visit_month,visit_day_of_week,hit_month,hit_day_of_week,utm_source_BHcvLfOaCWvWTykYqHVe,utm_source_MvfHsxITijuriZxsqZqt,utm_source_QxAxdyPLuQMEcrdZWdWb,utm_source_RmEBuqrriAfAVsLQQmhk,utm_source_ZpYIoDJMcFzVoPFsHGJL,utm_source_aXQzDWsJuGXeBXexNHjc,...,geo_city_Krasnoyarsk,geo_city_Moscow,geo_city_Nizhny Novgorod,geo_city_Novosibirsk,geo_city_Saint Petersburg,geo_city_Samara,geo_city_Ufa,geo_city_Yekaterinburg,geo_city_other,target
0,11,2,11,2,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,11,2,11,2,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,11,6,11,6,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,12,1,12,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0
4,12,1,12,1,0,0,0,0,1,0,...,1,0,0,0,0,0,0,0,0,0


In [67]:
#контроль списка колонок итогового датафрейма
list(df.columns)

['visit_month',
 'visit_day_of_week',
 'hit_month',
 'hit_day_of_week',
 'utm_source_BHcvLfOaCWvWTykYqHVe',
 'utm_source_MvfHsxITijuriZxsqZqt',
 'utm_source_QxAxdyPLuQMEcrdZWdWb',
 'utm_source_RmEBuqrriAfAVsLQQmhk',
 'utm_source_ZpYIoDJMcFzVoPFsHGJL',
 'utm_source_aXQzDWsJuGXeBXexNHjc',
 'utm_source_bByPQxmDaMXgpHeypKSM',
 'utm_source_fDLlAcSmythWSCVMvqvL',
 'utm_source_jaSOmLICuBzCFqHfBdRg',
 'utm_source_kjsLglQLzykiRbcDiGcD',
 'utm_source_other',
 'utm_source_vFcAhRxLfOWKhvxjELkx',
 'device_category_desktop',
 'device_category_mobile',
 'device_category_tablet',
 'geo_country_Russia',
 'geo_country_other',
 'geo_city_Chelyabinsk',
 'geo_city_Kazan',
 'geo_city_Krasnodar',
 'geo_city_Krasnoyarsk',
 'geo_city_Moscow',
 'geo_city_Nizhny Novgorod',
 'geo_city_Novosibirsk',
 'geo_city_Saint Petersburg',
 'geo_city_Samara',
 'geo_city_Ufa',
 'geo_city_Yekaterinburg',
 'geo_city_other',
 'target']

## Modelling

In [68]:
#выделение целевого признака и разделение датафрейма на тренировочную и тестовую выборки для обучения моделей
x = df.drop(["target"], axis=1)
y = df["target"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [69]:
#обучение моделей со стандтартными гиперпараметрами
models = [
    CatBoostClassifier(verbose=False, random_state=42),
    LGBMClassifier(random_state=42),
    MLPClassifier(random_state=42),
    RandomForestClassifier(n_jobs=-1, random_state=42),
    XGBClassifier(random_state=42)
]

for m in models:
    m.fit(x_train, y_train)
    score = roc_auc_score(y_test, m.predict_proba(x_test)[:, 1])
    print(f"{type(m).__name__}, roc_auc_score: {score}")

CatBoostClassifier, roc_auc_score: 0.6911226864185978
LGBMClassifier, roc_auc_score: 0.6836378827006673
MLPClassifier, roc_auc_score: 0.668905645424307
RandomForestClassifier, roc_auc_score: 0.6907509887355732
XGBClassifier, roc_auc_score: 0.6871907577363565


In [70]:
#обучение моделей с изменёнными гиперпараметрами
models_tun = [
    CatBoostClassifier(max_depth=10, verbose=False, random_state=42),
    LGBMClassifier(n_estimators=2000, random_state=42),
    RandomForestClassifier(max_features="log2", n_jobs=-1, random_state=42),
    XGBClassifier(n_estimators=2000, random_state=42)
]

best_score = 0
best_model = None
    
for model in models_tun:
    model.fit(x_train, y_train)
    score = roc_auc_score(y_test, model.predict_proba(x_test)[:, 1])
    if score > best_score:
        best_score = score
        best_model = model
    print(f"{type(model).__name__}, roc_auc_score: {score}")

CatBoostClassifier, roc_auc_score: 0.6915128635058602
LGBMClassifier, roc_auc_score: 0.69356989866844
RandomForestClassifier, roc_auc_score: 0.6907509887355732
XGBClassifier, roc_auc_score: 0.6918520203449955


### Results

In [71]:
#обучение лучшей модели на всём датафрейме
best_model.fit(x, y)

LGBMClassifier(n_estimators=2000, random_state=42)

In [73]:
#сериализация лучшей обученной модели в файл
with open("model.pkl", "wb") as file:
    dill.dump({
        "model": best_model,
        "metadata": {
            "name": "DS Intro. Final Project",
            "author": "Dmitrii Kolesnik",
            "version": 1,
            "date": datetime.now(),
            "type": type(best_model).__name__,
            "roc_auc_score": round(best_score, 4)
    }
    }, file)